This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'TGHsoDYLQX8kgVycyuCN'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [39]:
# First, import the relevant modules
import requests
import collections
import statistics

In [8]:
print ("""https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-04-15
                             &end_date=2017-04-15&api_key="""+API_KEY)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-04-15
                             &end_date=2017-04-15&api_key=TGHsoDYLQX8kgVycyuCN


In [18]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
quand_request = requests.get( """https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-04-15
                             &end_date=2017-04-16&api_key=""" + API_KEY)

In [19]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print (quand_request.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-04-15', 'end_date': '2017-04-16', 'frequency': 'daily', 'data': [], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [13]:
year_request = requests.get ("""https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01
                             &end_date=2017-12-31&api_key=""" + API_KEY)

### 2. Convert the returned JSON object into a Python dictionary.

In [21]:
# Returned as a python dictionary
year_dict = year_request.json()
print (type(year_dict))

<class 'dict'>


### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [59]:
#The opening  price occurs at [1] of each table in the data
#Assign the data dictionary to the varaible data and show the first 5 days
data = year_dict['dataset_data']['data']
print (data[0:5])

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]]


In [31]:
#Set an arbitriary value for the max and min that we know is somewhere in the middle of the possible ranges. 
#Also accomodate for the days where no open prices are available 
max_open = 0
min_open = 50
for day in data:
    if day[1] == None:
        pass
    elif day[1] > max_open:
        max_open = day[1]
    elif day[1] < min_open:
        min_open = day[1]
print ('Highest opening price:', max_open)
print ('Lowest opening price:', min_open)

Highest opening price: 53.11
Lowest opening price: 34.0


### 4. What was the largest change in any one day (based on High and Low price)?

In [32]:
#High occurs at [2] and low occurs at [3]
max_change = 0
for day in data:
    if (day[2] - day[3]) > max_change:
        max_change = (day[2] - day[3])
        
print ('Highest change price:', max_change)

Highest change price: 2.8100000000000023


### 5. What was the largest change between any two days (based on Closing Price)?

In [36]:
#Largest price change occurs at [4]
largest_two_days = 0
for day in range(0, len(data)-1):
    two_days = abs(data[day][4] - data[day+1][4])
    if two_days > largest_two_days:
        largest_two_days = two_days
print ('Largest price change between two days:', largest_two_days)

Largest price change between two days: 2.559999999999995


### 6. What was the average daily trading volume during this year?

In [41]:
#Trading volume occurs at [6]
#Using the statistics library
average_trading = statistics.mean([day[6] for day in data])
print ('Average daily trading volume for 2017:', average_trading)

Average daily trading volume for 2017: 89124.33725490196


In [45]:
#Using a self-made average calculatoion
avg_trading = sum([day[6] for day in data]) / len(data)
print ('Average daily trading volume for 2017:', avg_trading)

89124.33725490196


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [46]:
#Using the statisics library
median_trading_volume = statistics.median([day[6] for day in data])
print ('Median trading volume for 2017:', median_trading_volume)

Median trading volume for 2017: 76286.0


In [55]:
print (len([day[6] for day in data]))

255


In [58]:
#Length of data is 255, which means the median occurs at day=128
#The answer using the median function of the statistics library occurs at day=127
median_trading = sorted([day[6] for day in data])[128]
print ('Median trading volume for 2017:', median_trading)

Median trading volume for 2017: 76600.0
